<a href="https://colab.research.google.com/github/Megaexoplanet/project-3/blob/master/PThree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Basic libaries
import numpy as np
from datetime import datetime as dt, timedelta as td
# Data manipulation
import pandas as pd
import csv
# Modelling
import statsmodels.api as sm
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler
import scipy as sp
import tensorflow as tf
# Visualizations
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
matplotlib.use('SVG')

In [ ]:
# Changing Frequency
freq=td(hours=1)
# Choose missing_data_handling to be True to fill missing data points with a concave function and False to remove missing data points.
missing_data_handling=False

In [ ]:
# reading in stock index date
prices=pd.read_csv("prices_60m.csv",parse_dates=['Dates'])
prices.rename(columns={'Dates':'dates'},inplace=True)
prices=prices.set_index('dates')
prices=prices.dropna(how='any')
prices

if missing_data_handling:
    prices=prices.resample(freq,base=15.5).asfreq()
    
# calculating logreturns and Z Scores
def logreturn(x):
    return np.log(x/x.shift(1))

def zscore(x):
    return (x-x.mean())/x.std()

prices['DJIA LogRet']=logreturn(prices['DJIA CLOSE'])
prices['SP500 LogRet']=logreturn(prices['SP500 CLOSE'])
prices['DJIA Z Scores']=zscore(prices['DJIA CLOSE'])
prices['SP500 Z Scores']=zscore(prices['SP500 CLOSE'])
prices['DJIA Log']=np.log(prices['DJIA CLOSE'])
prices['SP500 Log']=np.log(prices['SP500 CLOSE'])
prices=prices.dropna(how='any')
prices

,DJIA CLOSE,SP500 CLOSE,DJIA LogRet,SP500 LogRet,DJIA Z Scores,SP500 Z Scores,DJIA Log,SP500 Log
dates,,,,,,,,
2019-06-21 17:30:00,26791.51,2952.75,-0.001901,-0.001739,0.241810,-0.054862,10.195840,7.990492
2019-06-21 18:30:00,26796.05,2953.07,0.000169,0.000108,0.250797,-0.048422,10.196010,7.990601
2019-06-21 19:30:00,26790.67,2953.51,-0.000201,0.000149,0.240148,-0.039568,10.195809,7.990750
2019-06-21 20:30:00,26793.13,2955.41,0.000092,0.000643,0.245017,-0.001331,10.195901,7.991393
2019-06-21 21:30:00,26719.13,2950.46,-0.002766,-0.001676,0.098545,-0.100948,10.193135,7.989716
...,...,...,...,...,...,...,...,...
2019-09-27 17:30:00,26890.71,2970.09,-0.004136,-0.004612,0.438163,0.294099,10.199536,7.996348
2019-09-27 18:30:00,26825.86,2961.04,-0.002415,-0.003052,0.309801,0.111971,10.197122,7.993296
2019-09-27 19:30:00,26776.07,2954.41,-0.001858,-0.002242,0.211249,-0.021455,10.195264,7.991054


In [ ]:
# Reading in csv file containing tweets and converting it to a pandas dataframe
tweets=pd.read_csv("raw_tweets.csv",parse_dates=['created_at'])
tweets = tweets.drop(columns="id")

# Converting time from UTC to CEST +2 hours
cest_date=[]
for date in tweets['created_at']:
    cest_date.append(date+td(hours=2))
tweets['created_at']=cest_date

# Changing index to Datetimeindex
tweets.rename(columns={'created_at':'date'},inplace=True)
tweets=tweets.set_index('date')
#tweets.index = pd.DatetimeIndex(tweets.index)

tweets

,tweet_text,bullish
date,,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession ri...,NaN
2019-06-24 19:58:00,@nyika_yedu The multi currency regime dealt wi...,NaN
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying ...,NaN
2019-06-24 19:58:00,"The caution continues. ""Small business owners ...",NaN
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - htt...,NaN
...,...,...
2019-09-03 20:47:00,@AmandaAgatiPNC @NFIB Are you a recession deni...,NaN
2019-09-03 20:47:00,NaN,True
2019-09-03 20:47:00,@pdtugs @CPC_HQ Actually Canada economy is goo...,NaN


In [ ]:
len(tweets)

1048575

In [ ]:
tweets['Time']=tweets.index.hour

In [ ]:
df=tweets.groupby(tweets.index.date).count()
df['Total']=df['tweet_text']+df['bullish']
df['Total'].mean()

14563.541666666666

In [ ]:
#tweets['Time'].value_counts().sort_index().plot.bar()
fig, ax = plt.subplots()
counts, bins, patches = ax.hist(tweets.Time, facecolor='gray', edgecolor='black', bins=range(0,25))
ax.set_xticks(bins)
ax.set_title('Tweetek óránkénti eloszlása', fontsize=9)
ax.tick_params(axis='both', which='major', labelsize=7)
plt.savefig("hourly_tweet_distribution.svg", format="svg")
print(counts)
print(bins)

[43588. 38517. 37106. 34865. 33845. 31385. 28502. 25502. 28511. 31300.
 29118. 29834. 36889. 41597. 55131. 61533. 63697. 62545. 61488. 56832.
 55641. 54242. 57459. 49448.]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]


In [ ]:
if not missing_data_handling:
    # Removing tweets that were created on weekends and bank holidays
    #getting trading dates from DJIA intraday prices
    t_dates=prices.index.map(pd.Timestamp.date).unique()
    str_t_dates=[str(x) for x in t_dates]
    tweets=tweets[tweets.index.floor('D').isin(str_t_dates)]
    # Removing tweets that are not in trading analysis interval
    start=str(td(hours=15,minutes=30)-freq)
    end=str(td(hours=21,minutes=29))
    tweets = tweets.between_time(start, end, include_start=True, include_end=True)

In [ ]:
# Separating sentiment tweets from bullish/bearish index tweets
text_tweets=tweets.loc[:, ['tweet_text']]
market_tweets=tweets.loc[:, ['bullish']]

In [ ]:
# Filtering out NaNs
fmarket_tweets=market_tweets[market_tweets['bullish'].notnull()]
ftext_tweets=text_tweets[text_tweets['tweet_text'].notnull()]
ftext_tweets =ftext_tweets.drop_duplicates(subset="tweet_text",keep=False)
print('Total number of observations including both text and bullish/bearish tweets:',ftext_tweets.tweet_text.count()+fmarket_tweets.bullish.count())
ftext_tweets

Total number of observations including both text and bullish/bearish tweets: 327195


,tweet_text
date,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession ri...
2019-06-24 19:58:00,@nyika_yedu The multi currency regime dealt wi...
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying ...
2019-06-24 19:58:00,"The caution continues. ""Small business owners ..."
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - htt...
...,...
2019-09-03 20:47:00,"@PrimeXbt In general, the SP500 is clearly ove..."
2019-09-03 20:47:00,@AmandaAgatiPNC @NFIB Are you a recession deni...
2019-09-03 20:47:00,@pdtugs @CPC_HQ Actually Canada economy is goo...


In [ ]:
text = " ".join(tweet for tweet in ftext_tweets.tweet_text)
print ('There are %s words in the combination of all review.' % (len(text)))

There are 49648007 words in the combination of all review.


In [ ]:
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords_raw=["one","co","amp","https","bullish", "bearish", "stock market", "I think economy", "I feel economy", "I am feeling economy", "It feels economy","inflation", "unemployment rate", "recession","SP500","S&P500","DJIA","Dow jones"]
stopwords_splited=[]
for phrase in stopwords_raw:
    split=phrase.split()
    stopwords_splited=list(set(stopwords_splited+split))
stopwords.update(stopwords_splited)

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white",width=800, height=400).generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()
fig = plt.gcf() #get current figure
#fig.set_size_inches(10,10)
#plt.savefig("tweets_word_cloud.png", format="png")

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Matplotlib is currently using svg, which is a non-GUI backend, so cannot show the figure.
  app.launch_new_instance()


In [ ]:
pd.set_option('display.max_colwidth', -1)
ftext_tweets[ftext_tweets['tweet_text'].str.contains("now")].sample(10)

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,tweet_text
date,
2019-07-10 16:26:00,"@JohnSnow2019 I have cash and buying power on hand to handle it. Sharpened all my tools throughout the years where I think I would make off pretty well. I have portfolio margin, can trade futures, options, volatility. Been waiting for one since the last Great Recession."
2019-08-15 15:22:00,@WSJ They absolutely should...i know in this country we feel we can do and say whatever we want..but why should they be allowed into a country they wanted to hurt the economy on with their bds movement...words are words but when you vote to boycott the countries business you are not
2019-08-12 17:57:00,@Pricd05 @ChineseSnowden @DEAURANGZEB @godeepak217 @MeRadhiika Lets see... i feel khaleel is the one who gets kicked on the teeth everytime i mean see his economy ...but he manages to take wickets somehow
2019-08-14 16:21:00,"@realDonaldTrump Sorry to see the stock market falling. My 401K is suffering. But the good news is this reduces your only claim to fame, even though actually Obama is due the credit. This ensures you will lose in 2020 and likely go to prison. So thanks to you, I now smile when I see my 401K fall."
2019-06-27 17:09:00,"RT acorio ""To help you get a quick look into servicenow, here are three key stock market metrics and their definitions. https://t.co/itDT9gWnDb #ServiceNow"""
2019-08-15 15:26:00,@catherineVsnow @liberalincincy @RepDonBeyer Stock market we are not the world police?? But keep up those record high numbers.. Thanks Trump
2019-07-10 20:47:00,@Jonath8888Lewis @acnewsitics @realDonaldTrump What exactly had he done?\nDon't say the economy because he inherited that. Don't say unemployment because I know people in positions that told me they aren't extending it so people just can't collect. Also Trump doesn't control the stock market. The stock holders do.
2019-08-29 19:01:00,Is a global recession now the best chance of forcing a resolution to the escalating trade conflict between the United States and China? https://t.co/UbenuQaDr3 https://t.co/M9pDj0XJED
2019-08-20 16:52:00,"@Jim_Jordan No. Down turns happen. It is cyclical. You know that. Having a plan to combat a recession is important. ""Marketing"" how great the economy does not constitute a plan. I know he's your guy but try to remember when truth, fundamentals and integrity meant something. Wish for honesty."


In [ ]:
# import nltk; nltk.download('popular')
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sia=SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...


In [ ]:
del sia.lexicon['thank']
del sia.lexicon['thanks']

In [ ]:
stock_lex = pd.read_csv('stock_lex.csv')
stock_lex['sentiment'] = (stock_lex['Aff_Score'] + stock_lex['Neg_Score'])/2
stock_lex = dict(zip(stock_lex.Item, stock_lex.sentiment))
#filtering out multiple words expressions from the lexicon
stock_lex = {k:v for k,v in stock_lex.items() if len(k.split(' '))==1}
stock_lex_scaled = {}
for k, v in stock_lex.items():
    if v > 0:
        stock_lex_scaled[k] = v / max(stock_lex.values()) * 4
    else:
        stock_lex_scaled[k] = v / min(stock_lex.values()) * -4

In [ ]:
positive = []
with open('lm_positive.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        positive.append(row[0].strip())
        
negative = []
with open('lm_negative.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        entry = row[0].strip().split(" ")
        if len(entry) > 1:
            negative.extend(entry)
        else:
            negative.append(entry[0])

In [ ]:
final_lex = {}
final_lex.update({word:2.0 for word in positive})
final_lex.update({word:-2.0 for word in negative})
final_lex.update(stock_lex_scaled)
final_lex.update(sia.lexicon)
sia.lexicon = final_lex

In [ ]:
final_lex

{'ABLE': 2.0,
 'ABUNDANCE': 2.0,
 'ABUNDANT': 2.0,
 'ACCLAIMED': 2.0,
 'ACCOMPLISH': 2.0,
 'ACCOMPLISHED': 2.0,
 'ACCOMPLISHES': 2.0,
 'ACCOMPLISHING': 2.0,
 'ACCOMPLISHMENT': 2.0,
 'ACCOMPLISHMENTS': 2.0,
 'ACHIEVE': 2.0,
 'ACHIEVED': 2.0,
 'ACHIEVEMENT': 2.0,
 'ACHIEVEMENTS': 2.0,
 'ACHIEVES': 2.0,
 'ACHIEVING': 2.0,
 'ADEQUATELY': 2.0,
 'ADVANCEMENT': 2.0,
 'ADVANCEMENTS': 2.0,
 'ADVANCES': 2.0,
 'ADVANCING': 2.0,
 'ADVANTAGE': 2.0,
 'ADVANTAGED': 2.0,
 'ADVANTAGEOUS': 2.0,
 'ADVANTAGEOUSLY': 2.0,
 'ADVANTAGES': 2.0,
 'ALLIANCE': 2.0,
 'ALLIANCES': 2.0,
 'ASSURE': 2.0,
 'ASSURED': 2.0,
 'ASSURES': 2.0,
 'ASSURING': 2.0,
 'ATTAIN': 2.0,
 'ATTAINED': 2.0,
 'ATTAINING': 2.0,
 'ATTAINMENT': 2.0,
 'ATTAINMENTS': 2.0,
 'ATTAINS': 2.0,
 'ATTRACTIVE': 2.0,
 'ATTRACTIVENESS': 2.0,
 'BEAUTIFUL': 2.0,
 'BEAUTIFULLY': 2.0,
 'BENEFICIAL': 2.0,
 'BENEFICIALLY': 2.0,
 'BENEFIT': 2.0,
 'BENEFITED': 2.0,
 'BENEFITING': 2.0,
 'BENEFITTED': 2.0,
 'BENEFITTING': 2.0,
 'BEST': 2.0,
 'BETTER': 2.0,
 'BOL

In [ ]:
sentiment=[]
for tweet in ftext_tweets['tweet_text']:
    sentiment.append(sia.polarity_scores(tweet)['compound'])
ftext_tweets['Sentiment Score']=sentiment

In [ ]:
pd.set_option('display.max_colwidth', -1)
ftext_tweets[ftext_tweets['tweet_text'].str.len()<150].sample(10)

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,tweet_text,Sentiment Score
date,,
2019-08-12 15:46:00,@realDonaldTrump hows the Dow Jones doing now fuck tard?,-0.7878
2019-08-29 15:12:00,"@realDonaldTrump The stock market is not the Economy! Read a book, Donald.",0.2155
2019-07-16 21:04:00,Economic Report: Import prices post biggest drop in June in six months; more evidence of falling inflation https://t.co/TiSXgyySjB,-0.4381
2019-08-14 15:22:00,YIELD CURVE INVERTS: Recession indicator flashes red for first time since 2005 https://t.co/NSDrjaqp14,-0.4861
2019-06-28 16:34:00,#volkswagon news from @Autoblog- VW's Traton truck unit stalls on stock market debut https://t.co/MG3NkAOpNP,0.6820
2019-08-15 15:36:00,When this president tweets the stock market loses money. He panic's traders with his spastic mind set.,0.1921
2019-07-01 17:40:00,@PlanetPonzi Nice Head &amp; Shoulders formation. Wonder how long the Stock Buybacks can elevate the market.,0.7336
2019-08-21 18:51:00,#KingOfIsrael is what #Trump is blurting out now so we lose focus on the trade war that is bringing a recession.,-0.8031
2019-07-18 15:43:00,@realDonaldTrump full margin seeeeeelllllll dow jones perfect perfect,0.8627


In [ ]:
# Downsample the tweet sentiment scores
iss = ftext_tweets['Sentiment Score'].resample(freq,base=15.5,label='right').mean().to_frame()
iss = iss[iss['Sentiment Score'].notnull()]
iss['Z Scores']=(iss['Sentiment Score']-iss['Sentiment Score'].mean())/iss['Sentiment Score'].std()

In [ ]:
iss

,Sentiment Score,Z Scores
date,,
2019-06-24 20:30:00,0.063693,0.105390
2019-06-24 21:30:00,0.132368,0.803242
2019-06-25 15:30:00,0.172388,1.209915
2019-06-25 16:30:00,0.198150,1.471695
2019-06-25 17:30:00,0.132584,0.805442
...,...,...
2019-09-03 17:30:00,-0.055798,-1.108834
2019-09-03 18:30:00,-0.010625,-0.649796
2019-09-03 19:30:00,0.027969,-0.257626


In [ ]:
ftext_tweets

,tweet_text,Sentiment Score
date,,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession risk.\nThis is my FT Alphachat podcast. Plus my views on Facebook's Libra coin \n https://t.co/QpRRmEetk4,-0.0400
2019-06-24 19:58:00,"@nyika_yedu The multi currency regime dealt with inflation, when the exchange rate is changing 3 times a day retailers are better not stocking than lose value on their stock. The Gvt is going to print whenever they are cornered &amp; rates will skyrocket",0.7282
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying for their loans and cannot get out from under them. It is a systemic problem for a generation that graduated at the time of the Great Recession. Unless you are a buying stocks in huge quantities YOU personally are not paying for anything. Read his plan,0.7321
2019-06-24 19:58:00,"The caution continues. ""Small business owners have been conservative about hiring since the recession. A hoped-for boom in small company hiring never happened because owners haven't wanted to take on the added risk and expense of more staffers.""\n\n https://t.co/jwebktIhA8",-0.0587
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - https://t.co/NVZd1jCmI1\n\n$SPY $SPX $ES_F $QQQ $NQ_F $DIA $DJIA $TLT $ZN_F $ZB_F,-0.4379
...,...,...
2019-09-03 20:47:00,"@PrimeXbt In general, the SP500 is clearly overrated https://t.co/YOzStZn4Gu",0.4051
2019-09-03 20:47:00,@AmandaAgatiPNC @NFIB Are you a recession denier? The bond market is screaming recession. I don't think it is different this time...,-0.8789
2019-09-03 20:47:00,"@pdtugs @CPC_HQ Actually Canada economy is good, American is headed to a recession thanks to Trump. Once again, good stewards of the economy, Liberals.",0.5850


make table with three fields: Date (day), volume of tweets, bullish percentage

In [ ]:
cleaned_tweets = ftext_tweets

In [ ]:
# cleaned_tweets['day'] = str(cleaned_tweets.index)[:10]
str(cleaned_tweets.index[x])[:10]

'2019-09-03'

In [ ]:
# clean_tweets = ftext_tweets["date"] = 
# filter into days
days = []

for x in range(cleaned_tweets.count()[0]):
    days.append(str(cleaned_tweets.index[x])[:10])
cleaned_tweets['day'] = days
cleaned_tweets.head()

,tweet_text,Sentiment Score,day,sentiment_boolean
date,,,,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession risk.\nThis is my FT Alphachat podcast. Plus my views on Facebook's Libra coin \n https://t.co/QpRRmEetk4,-0.0400,2019-06-24,0
2019-06-24 19:58:00,"@nyika_yedu The multi currency regime dealt with inflation, when the exchange rate is changing 3 times a day retailers are better not stocking than lose value on their stock. The Gvt is going to print whenever they are cornered &amp; rates will skyrocket",0.7282,2019-06-24,1
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying for their loans and cannot get out from under them. It is a systemic problem for a generation that graduated at the time of the Great Recession. Unless you are a buying stocks in huge quantities YOU personally are not paying for anything. Read his plan,0.7321,2019-06-24,1
2019-06-24 19:58:00,"The caution continues. ""Small business owners have been conservative about hiring since the recession. A hoped-for boom in small company hiring never happened because owners haven't wanted to take on the added risk and expense of more staffers.""\n\n https://t.co/jwebktIhA8",-0.0587,2019-06-24,0
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - https://t.co/NVZd1jCmI1\n\n$SPY $SPX $ES_F $QQQ $NQ_F $DIA $DJIA $TLT $ZN_F $ZB_F,-0.4379,2019-06-24,0


In [ ]:
cleaned_tweets.head()

,tweet_text,Sentiment Score,day
date,,,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession risk.\nThis is my FT Alphachat podcast. Plus my views on Facebook's Libra coin \n https://t.co/QpRRmEetk4,-0.0400,2019-06-24
2019-06-24 19:58:00,"@nyika_yedu The multi currency regime dealt with inflation, when the exchange rate is changing 3 times a day retailers are better not stocking than lose value on their stock. The Gvt is going to print whenever they are cornered &amp; rates will skyrocket",0.7282,2019-06-24
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying for their loans and cannot get out from under them. It is a systemic problem for a generation that graduated at the time of the Great Recession. Unless you are a buying stocks in huge quantities YOU personally are not paying for anything. Read his plan,0.7321,2019-06-24
2019-06-24 19:58:00,"The caution continues. ""Small business owners have been conservative about hiring since the recession. A hoped-for boom in small company hiring never happened because owners haven't wanted to take on the added risk and expense of more staffers.""\n\n https://t.co/jwebktIhA8",-0.0587,2019-06-24
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - https://t.co/NVZd1jCmI1\n\n$SPY $SPX $ES_F $QQQ $NQ_F $DIA $DJIA $TLT $ZN_F $ZB_F,-0.4379,2019-06-24


In [ ]:
# get average sentiment score for each day
x = cleaned_tweets.groupby('day', as_index=False)['Sentiment Score'].mean()
# get total number of tweets for each day
y = cleaned_tweets.groupby('day', as_index=False)['tweet_text'].count()
# percentage bullish


In [ ]:
sentiment_boolean = []
for y in range(cleaned_tweets.count()[0]):
    if cleaned_tweets['Sentiment Score'][y] >= 0:
        sentiment_boolean.append(1)
    else:
        sentiment_boolean.append(0)
cleaned_tweets['sentiment_boolean'] = sentiment_boolean
cleaned_tweets.head()

,tweet_text,Sentiment Score,day,sentiment_boolean
date,,,,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession risk.\nThis is my FT Alphachat podcast. Plus my views on Facebook's Libra coin \n https://t.co/QpRRmEetk4,-0.0400,2019-06-24,0
2019-06-24 19:58:00,"@nyika_yedu The multi currency regime dealt with inflation, when the exchange rate is changing 3 times a day retailers are better not stocking than lose value on their stock. The Gvt is going to print whenever they are cornered &amp; rates will skyrocket",0.7282,2019-06-24,1
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying for their loans and cannot get out from under them. It is a systemic problem for a generation that graduated at the time of the Great Recession. Unless you are a buying stocks in huge quantities YOU personally are not paying for anything. Read his plan,0.7321,2019-06-24,1
2019-06-24 19:58:00,"The caution continues. ""Small business owners have been conservative about hiring since the recession. A hoped-for boom in small company hiring never happened because owners haven't wanted to take on the added risk and expense of more staffers.""\n\n https://t.co/jwebktIhA8",-0.0587,2019-06-24,0
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - https://t.co/NVZd1jCmI1\n\n$SPY $SPX $ES_F $QQQ $NQ_F $DIA $DJIA $TLT $ZN_F $ZB_F,-0.4379,2019-06-24,0


In [ ]:
# percentage bullish
z = cleaned_tweets.groupby('day', as_index=False)['sentiment_boolean'].sum() # number of bullish tweets on the day
z_list = list(z['sentiment_boolean'])
y_list = list(y['tweet_text'])

In [ ]:
z_list[0]/y_list[0]

0.5625

In [ ]:
z_percent = []
for i in range(len(z_list)):
    z_percent.append(z_list[i]/y_list[i])
z_percent

[0.5625,
 0.6272787173292335,
 0.622778382299601,
 0.6036036036036037,
 0.665938864628821,
 0.6409415971394518,
 0.6696546269443712,
 0.6134146341463415,
 0.5715919487648673,
 0.6070469798657718,
 0.5740861618798956,
 0.5643111337061515,
 0.5536919221129748,
 0.6349892008639308,
 0.6074332171893148,
 0.5831024930747922,
 0.5983115468409586,
 0.5733678086619263,
 0.5982982534706672,
 0.5803033951360462,
 0.6181520850367948,
 0.6318021201413427,
 0.5574288724973656,
 0.5560514570753479,
 0.5658279695728496,
 0.5724310776942356,
 0.5221206581352834,
 0.5589340621797062,
 0.5428015564202334,
 0.5361503856041131,
 0.540333660451423,
 0.5231900452488688,
 0.5363272983776158,
 0.48408746481922493,
 0.49032258064516127,
 0.5435852881552273,
 0.3734301709789681,
 0.4145185185185185,
 0.4377851769638673,
 0.38443766749838276,
 0.40302013422818794,
 0.42262501138537206,
 0.4492753623188406,
 0.47636717024472125,
 0.5093916939514045,
 0.4559244901398955,
 0.48754318967084925,
 0.4980671414038657,


In [ ]:
tweets_summary = x

In [ ]:
tweets_summary['Volume'] = y_list
tweets_summary['BullishDecimal'] = z_percent
tweets_summary['TotalBullish'] = z_list

In [ ]:
# tweets_summary.columns = ['Day', 'Average Sentiment Score', 'Volume', 'BullishDecimal', 'TotalBullish']
# sum(list(tweets_summary['Volume']))
tweets_summary.index.tolist()

['2019-06-24',
 '2019-06-25',
 '2019-06-26',
 '2019-06-27',
 '2019-06-28',
 '2019-07-01',
 '2019-07-02',
 '2019-07-03',
 '2019-07-05',
 '2019-07-08',
 '2019-07-09',
 '2019-07-10',
 '2019-07-11',
 '2019-07-12',
 '2019-07-15',
 '2019-07-16',
 '2019-07-17',
 '2019-07-18',
 '2019-07-19',
 '2019-07-22',
 '2019-07-23',
 '2019-07-24',
 '2019-07-25',
 '2019-07-26',
 '2019-07-29',
 '2019-07-30',
 '2019-07-31',
 '2019-08-01',
 '2019-08-02',
 '2019-08-05',
 '2019-08-06',
 '2019-08-07',
 '2019-08-08',
 '2019-08-09',
 '2019-08-12',
 '2019-08-13',
 '2019-08-14',
 '2019-08-15',
 '2019-08-16',
 '2019-08-19',
 '2019-08-20',
 '2019-08-21',
 '2019-08-22',
 '2019-08-23',
 '2019-08-26',
 '2019-08-27',
 '2019-08-28',
 '2019-08-29',
 '2019-08-30',
 '2019-09-03']

In [ ]:
cleaned_prices = prices
days_p = []
times_p = []
for j in range(prices.count()[0]):
    days_p.append(str(cleaned_prices.index[j])[:10])
    times_p.append(str(cleaned_prices.index[j])[11:])
cleaned_prices['day'] = days_p
cleaned_prices['times'] = times_p

In [ ]:
cleaned_prices.tail()

,DJIA CLOSE,SP500 CLOSE,DJIA LogRet,SP500 LogRet,DJIA Z Scores,SP500 Z Scores,DJIA Log,SP500 Log,day,times
dates,,,,,,,,,,
2019-09-27 17:30:00,26890.71,2970.09,-0.004136,-0.004612,0.438163,0.294099,10.199536,7.996348,2019-09-27,17:30:00
2019-09-27 18:30:00,26825.86,2961.04,-0.002415,-0.003052,0.309801,0.111971,10.197122,7.993296,2019-09-27,18:30:00
2019-09-27 19:30:00,26776.07,2954.41,-0.001858,-0.002242,0.211249,-0.021455,10.195264,7.991054,2019-09-27,19:30:00
2019-09-27 20:30:00,26755.14,2952.05,-0.000782,-0.000799,0.169821,-0.068950,10.194482,7.990255,2019-09-27,20:30:00
2019-09-27 21:30:00,26820.25,2961.79,0.002431,0.003294,0.298697,0.127064,10.196912,7.993549,2019-09-27,21:30:00


In [ ]:
max_time = cleaned_prices['times'].max()
# Get names of indexes for which column times is not max time
non_closing_times = cleaned_prices[cleaned_prices['times'] != max_time ].index
# Delete these row indexes from dataFrame
cleaned_prices.drop(non_closing_times , inplace=True)

In [ ]:
# only keep days that match up with days from tweets_summary df
cleaned_prices.head()

,DJIA CLOSE,SP500 CLOSE,DJIA LogRet,SP500 LogRet,DJIA Z Scores,SP500 Z Scores,DJIA Log,SP500 Log,day,times
dates,,,,,,,,,,
2019-06-21 21:30:00,26719.13,2950.46,-0.002766,-0.001676,0.098545,-0.100948,10.193135,7.989716,2019-06-21,21:30:00
2019-06-24 21:30:00,26727.54,2945.35,-0.000733,-0.000597,0.115191,-0.203785,10.193450,7.987983,2019-06-24,21:30:00
2019-06-25 21:30:00,26548.22,2917.38,-0.000915,-0.001740,-0.239747,-0.766671,10.186718,7.978441,2019-06-25,21:30:00
2019-06-26 21:30:00,26536.82,2913.78,-0.001711,-0.000799,-0.262312,-0.839120,10.186288,7.977206,2019-06-26,21:30:00
2019-06-27 21:30:00,26526.58,2924.92,-0.001277,-0.000243,-0.282580,-0.614931,10.185903,7.981022,2019-06-27,21:30:00
...,...,...,...,...,...,...,...,...,...,...
2019-09-23 21:30:00,26949.99,2991.78,-0.001882,-0.002104,0.555499,0.730603,10.201738,8.003624,2019-09-23,21:30:00
2019-09-24 21:30:00,26807.77,2966.60,0.001399,0.000951,0.273995,0.223864,10.196447,7.995172,2019-09-24,21:30:00
2019-09-25 21:30:00,26970.71,2984.87,-0.000290,-0.000392,0.596511,0.591541,10.202507,8.001311,2019-09-25,21:30:00


In [ ]:
tweets_summary = tweets_summary.set_index('Day')
cleaned_prices = cleaned_prices.set_index('day')

In [ ]:
days_column = tweets_summary.index.tolist()
days_column.remove('2019-07-03')
len(days_column)

49

In [ ]:
result = pd.concat([cleaned_prices, tweets_summary], axis=1, join='inner')
result['day'] = days_column
result = result.reset_index(drop=True)
result.rename(columns={'day': 'Day'}, inplace=True)
result.to_csv(r"C:\Users\Owner\Desktop\Projects\stock_data.csv", header=True)
result

,DJIA CLOSE,SP500 CLOSE,DJIA LogRet,SP500 LogRet,DJIA Z Scores,SP500 Z Scores,DJIA Log,SP500 Log,times,Average Sentiment Score,Volume,BullishDecimal,TotalBullish,Day
0,26727.54,2945.35,-0.000733,-0.000597,0.115191,-0.203785,10.193450,7.987983,21:30:00,0.112120,736,0.562500,414,2019-06-24
1,26548.22,2917.38,-0.000915,-0.001740,-0.239747,-0.766671,10.186718,7.978441,21:30:00,0.162006,4553,0.627279,2856,2019-06-25
2,26536.82,2913.78,-0.001711,-0.000799,-0.262312,-0.839120,10.186288,7.977206,21:30:00,0.148589,2757,0.622778,1717,2019-06-26
3,26526.58,2924.92,-0.001277,-0.000243,-0.282580,-0.614931,10.185903,7.981022,21:30:00,0.127418,2664,0.603604,1608,2019-06-27
4,26599.96,2941.76,0.002416,0.003374,-0.137335,-0.276032,10.188665,7.986763,21:30:00,0.204657,4122,0.665939,2745,2019-06-28
5,26717.43,2964.33,0.001007,0.001303,0.095180,0.178181,10.193071,7.994406,21:30:00,0.166332,3356,0.640942,2151,2019-07-01
6,26786.68,2973.01,0.002528,0.002290,0.232250,0.352863,10.195660,7.997330,21:30:00,0.167932,3793,0.669655,2540,2019-07-02
7,26922.12,2990.41,-0.000214,-0.000254,0.500334,0.703032,10.200704,8.003166,21:30:00,0.099155,4372,0.571592,2499,2019-07-05
8,26806.14,2975.95,-0.000030,0.000013,0.270768,0.412029,10.196386,7.998319,21:30:00,0.141956,2980,0.607047,1809,2019-07-08
9,26783.49,2979.63,0.002118,0.002046,0.225936,0.486088,10.195541,7.999554,21:30:00,0.105030,3064,0.574086,1759,2019-07-09


In [ ]:
result.count()

DJIA CLOSE                 49
SP500 CLOSE                49
DJIA LogRet                49
SP500 LogRet               49
DJIA Z Scores              49
SP500 Z Scores             49
DJIA Log                   49
SP500 Log                  49
times                      49
Average Sentiment Score    49
Volume                     49
BullishDecimal             49
TotalBullish               49
Day                        49
dtype: int64

In [ ]:

#PIE CHART 
#posi=0
# nega=0
# for i in range (0,len(tweets_summary)):
#     get_val=tweets_summary['Sentiment Score'][i]
#     if(float(get_val)<(0)):
#         nega=nega+1
#     if(float(get_val>(0))):
#         posi=posi+1
# posper=(posi/(len(tweets_summary)))*100
# negper=(nega/(len(tweets_summary)))*100
# print("% of positive tweets= ",posper)
# print("% of negative tweets= ",negper)
# arr=np.asarray([posper,negper], dtype=int)
# mlpt.pie(arr,labels=['positive','negative'])
# mlpt.plot()

In [ ]:
#train_start_date = '2019-06-24'
#train_end_date = '2019-07-24'
#test_start_date = '2019-07-25'
#test_end_date = '2019-09-27'
train_start_index = '0'
train_end_index = '40'
test_start_index = '41'
test_end_index = '49'
#train = tweets_summary.loc[train_start_date : train_end_date]
#test = tweets_summary.loc[test_start_date:test_end_date]
train = result.loc[train_start_index : train_end_index]
test = result.loc[test_start_index:test_end_index]

In [ ]:
sentiment_score_list = []
for day, row in train.T.iteritems():
    sentiment_score = np.asarray([result.loc[day, 'Average Sentiment Score'],result.loc[day, 'Volume']])
    sentiment_score_list.append(sentiment_score)
numpy_df_train = np.asarray(sentiment_score_list)

In [ ]:
print(numpy_df_train)

[[ 1.12119701e-01  7.36000000e+02]
 [ 1.62005601e-01  4.55300000e+03]
 [ 1.48589300e-01  2.75700000e+03]
 [ 1.27417530e-01  2.66400000e+03]
 [ 2.04657181e-01  4.12200000e+03]
 [ 1.66331913e-01  3.35600000e+03]
 [ 1.67931795e-01  3.79300000e+03]
 [ 9.91554895e-02  4.37200000e+03]
 [ 1.41955772e-01  2.98000000e+03]
 [ 1.05030287e-01  3.06400000e+03]
 [ 8.77147941e-02  3.93400000e+03]
 [ 9.35516291e-02  5.18700000e+03]
 [ 1.60023419e-01  3.24100000e+03]
 [ 1.25249826e-01  3.44400000e+03]
 [ 1.06645346e-01  3.61000000e+03]
 [ 1.35793437e-01  3.67200000e+03]
 [ 1.07922409e-01  4.64100000e+03]
 [ 1.28626646e-01  4.46600000e+03]
 [ 1.18689477e-01  4.15300000e+03]
 [ 1.47888580e-01  3.66900000e+03]
 [ 1.65864099e-01  2.83000000e+03]
 [ 7.24528978e-02  3.79600000e+03]
 [ 7.44862431e-02  3.80900000e+03]
 [ 9.27133119e-02  3.41800000e+03]
 [ 8.34704762e-02  3.99000000e+03]
 [ 3.97744241e-02  5.47000000e+03]
 [ 7.47864708e-02  5.85400000e+03]
 [ 5.62448638e-02  5.14000000e+03]
 [ 3.03854756e-02  6

In [ ]:
sentiment_score_list = []
for day, row in test.T.iteritems():
    sentiment_score = np.asarray([result.loc[day, 'Average Sentiment Score'],result.loc[day, 'Volume']])
    sentiment_score_list.append(sentiment_score)
numpy_df_test = np.asarray(sentiment_score_list)

In [ ]:
print(numpy_df_test)

[[-7.31769189e-02  7.45200000e+03]
 [-3.91301175e-02  1.13190000e+04]
 [ 2.86525935e-03  5.80300000e+03]
 [-1.11055115e-02  5.93300000e+03]
 [-2.79378978e-02  5.49900000e+03]
 [-6.38724313e-03  4.91500000e+03]
 [-5.22750000e-02  6.57600000e+03]
 [-1.89644522e-03  5.14800000e+03]]


In [ ]:
y_train = pd.DataFrame(train['SP500 CLOSE'])
y_test = pd.DataFrame(test['SP500 CLOSE'])
# print(y_train)
# print(y_test)

[2945.35,
 2917.38,
 2913.78,
 2924.92,
 2941.76,
 2964.33,
 2973.01,
 2990.41,
 2975.95,
 2979.63,
 2993.07,
 2999.91,
 3013.77,
 3014.3,
 3004.04,
 2984.42,
 2995.11,
 2976.61,
 2985.03,
 3005.47,
 3019.56,
 3003.67,
 3025.86,
 3020.97,
 3013.18,
 2980.38,
 2953.56,
 2932.05,
 2844.74,
 2881.77,
 2883.98,
 2938.09,
 2918.65,
 2882.69,
 2926.32,
 2840.6,
 2847.6,
 2888.68,
 2923.65,
 2900.51,
 2924.43]

In [ ]:
from treeinterpreter import treeinterpreter as ti
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix
rf = RandomForestRegressor()
rf.fit(numpy_df_train, y_train)

from sklearn.neural_network import MLPClassifier
mlpc = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', #'relu', the rectified linear unit function
                     solver='lbfgs', alpha=0.005, learning_rate_init = 0.001, shuffle=False)
mlpc.fit(numpy_df_train, y_train['SP500 CLOSE'].tolist())
prediction = mlpc.predict(numpy_df_test)

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


ValueError: Unknown label type: (array([2945.35, 2917.38, 2913.78, 2924.92, 2941.76, 2964.33, 2973.01,
       2990.41, 2975.95, 2979.63, 2993.07, 2999.91, 3013.77, 3014.3 ,
       3004.04, 2984.42, 2995.11, 2976.61, 2985.03, 3005.47, 3019.56,
       3003.67, 3025.86, 3020.97, 3013.18, 2980.38, 2953.56, 2932.05,
       2844.74, 2881.77, 2883.98, 2938.09, 2918.65, 2882.69, 2926.32,
       2840.6 , 2847.6 , 2888.68, 2923.65, 2900.51, 2924.43]),)

In [ ]:
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

RandomForestRegressor(n_estimators=10, n_jobs=1)

Prediction

In [ ]:
prediction, bias, contributions = ti.predict(rf, numpy_df_test)
print(prediction)

[[2895.7427]
 [2906.6775]
 [2886.2169]
 [2887.2544]
 [2908.2739]
 [2891.0306]
 [2895.4677]
 [2886.9843]]


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
idx=np.arange(int(test_start_index),int(test_end_index))
predictions_df_ = pd.DataFrame(data=prediction[0:], index = idx, columns=['SP500 CLOSE'])

In [ ]:
ax = predictions_df_.rename(columns={"Prices": "predicted_price"}).plot(title='Random Forest predicted prices')#predicted value
ax.set_xlabel("Indexes")
ax.set_ylabel("Stock Prices")
fig = y_test.rename(columns={"Prices": "actual_price"}).plot(ax = ax).get_figure()#actual value
fig.savefig("random forest.png")

In [ ]:
# MLPC 
# print(mlpc.score(predictions_df_, train['SP500 CLOSE']))
# print(accuracy_score(test['adj_close_price'],predictions_df['predicted_price']))

NotFittedError: This MLPClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
# from sklearn.metrics import accuracy_score
# y_pred = [0, 2, 1, 3]
# y_true = [0, 1, 2, 3]
# accuracy_score(y_true, y_pred)
# 0.5
# accuracy_score(y_true, y_pred, normalize=False)